## fixeval의 파이썬 데이터를 다운
### 데이터 전처리
1. 테스트 케이스만 틀린 코드쌍만 남김 -> 논리적 오류를 제외한 오류는 코딩으로 해결가능
2. 문제 설명이 없는 문제["p02479","p02480","p02481","p02482","p02483","p02484","p02485","p02486","p02487","p02488","p02489","p02490","p02491","p02492","p02493","p02494","p02495","p02496","p02497","p02498","p02499","p02506","p02510","p02523","p02524","p02525","p02526","p02527","p02528","p02529","p02530","p02531","p02532"]의 코드쌍은 삭제 -> 문제 설명을 모델의 입력값으로 넣어야하기 때문에 
3. AST로 변환 및 재변환 -> 코드 표현 일반화 및 컴파일이 되지 않는 코드쌍 삭제
4. 412 token을 넘어가는 코드 쌍 삭제 -> 모델의 최대 토큰이 512이며 문제 설명도 넣어야 하기 때문에 코드의 토큰을 412로 제한
5. 문제당 코드의 수가 100미만인 코드 쌍 삭제 -> 문제에 따른 코드가 학습되지 않을 것, 어려운 문제라고 판단, 논리적 오류는 문제에 종속된 원인이 존재하기 때문
6. 문제 설명 토큰화 -> 토큰화가 가능한 문제 선별 -> atcoder 문제는 score가 300이하 문제로 선별 총 500여개의 문제 토큰화, \n \leq \times와 같은 문자 치환
7. nl+pl 토큰 생성 및 길이가 512를 넘는 코드쌍 삭제
8. 변수, 상수 추상화 및 인용구 추출 -> 코딩 문제 특성상 숫자, 문자 상수는 문제에 따라 의미가 많기 때문에 상수 추상화X, 변수 및 사용자정의 함수만 추상화

### 해결해야할 것
1. 코드 쌍 사이의 차이가 크면 학습에 악영향을 끼칠수 있다.
2. 문제의 입력값을 제대로 받지 못해서 생기는 오류 해결 -> 입력값을 제대로 입력받았는지를 따로 테스트하는 task를 채점 전에 수행

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import json
import os
from bs4 import BeautifulSoup as bs
import ast
tokenizer = AutoTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1")
JSON_LEN=116
DESC_PATH="./Project_CodeNet/problem_descriptions/"
CODE_PAIR_PATH="normal_jsons/"
EXTRACT_DESC_PATH="tmp_p_desc/"


/home/sjw/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


#### 1. 데이터셋 다운로드
- FixEval: Execution-based Evaluation of Program Fixes for Competitive Programming Problems 의 데이터 셋을 활용 
  * codenet의 데이터셋중에서 파이썬으로 작성한 코드만 뽑아내고 한 유저가 한가지 문제를 풀어내기까지의 코드 변화내역을 1:m으로 엮은 데이터셋   

1. https://drive.google.com/drive/folders/1dzuHuouuWzlFCy1CMj9DYG9JGraEay27 에서 python.zip 파일 다운로드 및 압축해제, 불필요한 폴더 제거


In [ ]:
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rjjYW8SB8f5Hr34ig84OKpNYOzdt03Ar' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1rjjYW8SB8f5Hr34ig84OKpNYOzdt03Ar" -O python.zip && rm -rf ~/cookies.txt

In [16]:

# 압축 해제 및 필요없는 폴더 삭제 
!unzip python.zip -x *.id *.python *.jsonl
!ln -s data/python/jsons jsons
!pwd
!ls jsons
JSON_LEN=116

#### 2. 테스트 케이스만 틀린 코드쌍을 제외한 코드쌍 제거
- 해당 프로젝트에서는 문제에 대한 논리적 오류를 고쳐주는 ai모델을 만들기 때문에 테스트케이스 미통과(WA)만 남기고 다른 것들은 제거한다.
- 필요없는 필드 제거
- 추가적으로 설명이 존재하지 않는 문제에 대한 코드쌍도 제거

In [ ]:
no_desc_id=["p02479","p02480","p02481","p02482","p02483","p02484","p02485","p02486","p02487","p02488","p02489","p02490","p02491","p02492","p02493","p02494","p02495","p02496","p02497","p02498","p02499","p02506","p02510","p02523","p02524","p02525","p02526","p02527","p02528","p02529","p02530","p02531","p02532"]
p_num=0

if not os.path.exists(CODE_PAIR_PATH):
    os.mkdir(CODE_PAIR_PATH)
for i in range(JSON_LEN):
    new_json=[]
    print(i)
    with open('jsons/'+str(i)+'.json', 'r') as f:
        json_data = json.load(f)
    for j in range(len(json_data)):
        if json_data[j]:
            if json_data[j][0]["verdict"]=="Wrong Answer" and (json_data[j][0]["problem_id"] not in no_desc_id):
                temp_code_pair=[]
                try:
                    for k in range(2):
                        node=ast.parse(json_data[j][k]["code_tokens"])
                        new_code=ast.unparse(node).replace("    ","\t")
                        temp_code_pair.append(
                            {
                                "problem_id": json_data[j][k]["problem_id"],
                                "code_tokens": new_code,
                                "submission_id": json_data[j][k]["submission_id"],
                                "verdict": json_data[j][k]["verdict"]
                            }
                        )
                except:
                        continue
                new_json.append(temp_code_pair)
                p_num+=1
    with open(CODE_PAIR_PATH+str(i)+'.json', 'w',encoding='utf-8') as mkf:
        json.dump(new_json, mkf, indent="\t")
print(p_num)
#653,742개의 코드쌍

#### 3. 초급자에 맞는 문제 선정 및 문제 설명 추출
- 코드쌍의 문제의 설명을 codenet 데이터셋에서 추출
- ai 모델의 입력값으로 같이 들어갈 예정

In [1]:
#다운로드 및 불필요한 폴더 제거
!wget https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/Project_CodeNet_metadata.tar.gz
!tar -xf Project_CodeNet_metadata.tar.gz
!rm -rf Project_CodeNet/metadata


--2023-05-13 22:23:21--  https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/Project_CodeNet_metadata.tar.gz
Resolving dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)... 23.33.53.122, 2600:140b:a800:5b2::d2d, 2600:140b:a800:586::d2d
Connecting to dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)|23.33.53.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 294521596 (281M) [application/x-gtar]
Saving to: ‘Project_CodeNet_metadata.tar.gz’

Project_CodeNet_met 100%[===================>] 280.88M  10.9MB/s    in 28s     

2023-05-13 22:23:51 (10.2 MB/s) - ‘Project_CodeNet_metadata.tar.gz’ saved [294521596/294521596]



In [3]:
#문제중 score가 400이상인 문제 삭제 및 설명부분 추출
from bs4 import BeautifulSoup as bs
import os

p_list=os.listdir(DESC_PATH)
if not os.path.exists(EXTRACT_DESC_PATH):
    os.mkdir(EXTRACT_DESC_PATH)
for i in p_list:
    flag=False
    if int(i.split(".")[0][1:])>=2479:
        with open(DESC_PATH+i) as html:
            soup = bs(html, "html.parser")
            print(i)
            try:
                elements = soup.var.text
                if int(elements)<=300:
                    text=""
                    for j in soup.find_all("section")[0].contents:
                        if j.text.lower()=="input":
                            break
                        text+=j.text
                    text=text.replace("Problem Statement","").strip()
                    with open(EXTRACT_DESC_PATH+i.split(".")[0]+".txt","w") as desc:
                        desc.write(text)
                else:
                    os.remove(DESC_PATH+i)
            except:
                text=""
                if soup.find_all("section"):
                    for j in soup.find_all("section")[0].contents:
                        if j.text.lower()=="input":
                            break
                        text+=j.text
                    text=text.replace("Problem Statement","").strip()
                    with open(EXTRACT_DESC_PATH+i.split(".")[0]+".txt","w") as desc:
                        desc.write(text)
    else:
        with open(DESC_PATH+i) as html:
            soup = bs(html, "html.parser")
            print(i)
            text=""
            for j in soup.contents:
                if j.text.lower()=="input":
                    break
                if j.name!="script":
                    text+=j.text
            text=text.split("Input")[0].replace("$","").strip()
            with open(EXTRACT_DESC_PATH+i.split(".")[0]+".txt","w") as desc:
                desc.write(text)

p00419.html
p03506.html
p01925.html
p00151.html
p01250.html
p00679.html
p00006.html
p01044.html
p03856.html
p03945.html
p03785.html
p03294.html
p01169.html
p00501.html
p02445.html
p00783.html
p01205.html
p00193.html
p00271.html
p00843.html
p03324.html
p00490.html
p01987.html
p00909.html
p00052.html
p01161.html
p01980.html
p01119.html
p01293.html
p00557.html
p00894.html
p02117.html
p00385.html
p01585.html
p00579.html
p03758.html
p01835.html
p01174.html
p00900.html
p02140.html
p01418.html
p03804.html
p02022.html
p00893.html
p00869.html
p03826.html
p00091.html
p02087.html
p02298.html
p01338.html
p00938.html
p01874.html
p04013.html
p01569.html
p01456.html
p00187.html
p00109.html
p01940.html
p01480.html
p02164.html
p00194.html
p01167.html
p00347.html
p02067.html
p00026.html
p01413.html
p00047.html
p00432.html
p00199.html
p02689.html
p00651.html
p00000.html
p00597.html
p01669.html
p01218.html
p00586.html
p03547.html
p03427.html
p00400.html
p00496.html
p00874.html
p00646.html
p00017.html
p009

In [ ]:
#특수기호 및 공백제거
import os
p_list=os.listdir(EXTRACT_DESC_PATH)
s_symbol={"≠":"!=","\dots":"...", "\cdot":"...", "\ldots":"...", "…":"...","\leq":"<=", "\le":"<=", "≤":"<=", "≦":"<=", "\times": "*","\ge":">=", "\geq":">=","\{": "{","\}": "{"  }
for i in p_list:
    p_text=""
    with open(EXTRACT_DESC_PATH+i,"r") as txt:
        for j in txt.readlines():
            if j=="\n":
                continue
            p_text+=j.rstrip()+"\n"
        for j in s_symbol.items():
            p_text=p_text.replace(j[0],j[1])
    with open(EXTRACT_DESC_PATH+i,"w") as txt:
        txt.write(p_text)

#### 4. 코드 쌍과 그에 맞는 문제 설명 토큰화 및 길이가 512가 넘는 코드쌍 제거
- codebert-small의 max_token_len은 512이므로 문제 설명과 입력 코드를 합한 전체 토큰이 512를 넘으면 안된다.

In [ ]:
!git clone https://github.com/tree-sitter/tree-sitter-python
!mkdir build

In [2]:
#토큰화에 필요한 tree_sitter 준비
from tree_sitter import Language, Parser
import re
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-python'
  ]
)
PY_LANGUAGE = Language('build/my-languages.so', 'python')
parser = Parser()
parser.set_language(PY_LANGUAGE)


In [ ]:
#코드쌍 추상화
def abstract_pl(code,ident):
    tree = parser.parse(bytes(code, "utf8"))
    root_node = tree.root_node
    def dfs(node):
        global method,var
        if node.child_count==0:
            if node.type=='identifier':
                if node.parent.type not in ["for_in_clause","call","attribute","dotted_name","except_clause"]:
                    if (not ident.get(node.text)) and (node.text not in [b'i',b'j',b'k',b'__name__',b'int',b'str',b'input',b'float']):
                        if node.parent.type=="function_definition":
                            ident[(node.text)]="method"+str(method)
                            method+=1
                        else:
                            ident[(node.text)]="var"+str(var)
                            var+=1
            return
        else:
            for i in range(node.child_count):
                dfs(node.children[i])
    dfs(root_node)
    for i in ident.items():
        code=re.sub(r'\b'+i[0].decode()+r'\b',i[1] , code)
    
    return code,ident
method=0
var=0
for i in range(JSON_LEN):
    print(i)
    new_json=[]
    with open(CODE_PAIR_PATH+str(i)+'.json', 'r') as f:
        json_data = json.load(f)
        for j in range(len(json_data)):
            code_pair=json_data[j]
            temp_ident={}
            method=0
            var=0
            for k in range(2):
                code=code_pair[k]["code_tokens"]
                new_code,temp_ident=abstract_pl(code,temp_ident)
                code_pair[k]["code_tokens"]=new_code
            new_json.append(code_pair)
        with open(CODE_PAIR_PATH+str(i)+'.json', 'w',encoding='utf-8') as mkf:
            json.dump(new_json, mkf, indent="\t")
                

In [ ]:
#nl과 pl 토큰 길이 추출
removed_num=0
code_num=0
desc_list=os.listdir(EXTRACT_DESC_PATH)
for i in range(len(desc_list)):
    desc_list[i]=desc_list[i].split(".")[0]
if not os.path.exists(CODE_PAIR_PATH):
    os.mkdir(CODE_PAIR_PATH)
for i in range(JSON_LEN):
    new_json=[]
    print(i)
    with open(CODE_PAIR_PATH+str(i)+'.json', 'r') as f:
        json_data = json.load(f)
        for j in range(len(json_data)):
            code_pair=json_data[j]
            not_compile=False
            for k in range(2):
                code=code_pair[k]["code_tokens"]
                code_token=tokenizer.tokenize(code)
                nl_token=[]
                if code_pair[k]["problem_id"] in desc_list:
                    with open(EXTRACT_DESC_PATH+code_pair[k]["problem_id"]+".txt",'r') as desc:
                        nl_token=tokenizer.tokenize(desc.read())
                        code_pair[k]["nl_pl_token"]=len(code_token)+len(nl_token)
                        if len(code_token)+len(nl_token)>512-3:
                            not_compile=True
                else:
                    not_compile=True
            if not_compile:
                removed_num+=1
                continue
            else:
                new_json.append(code_pair)
                code_num+=1
    with open(CODE_PAIR_PATH+str(i)+'.json', 'w',encoding='utf-8') as mkf:
        json.dump(new_json, mkf, indent="\t")
print(removed_num,code_num)

In [10]:
#문제별 코드 수 
temp={}
for i in range(JSON_LEN):
    with open(CODE_PAIR_PATH+str(i)+'.json', 'r') as f:
        json_data = json.load(f)
        for j in range(len(json_data)):
            p_id=json_data[j][0]["problem_id"]
            try:
                temp[p_id]+=1
            except:
                temp[p_id]=1
with open('nl_pl_p_num.json', 'w',encoding='utf-8') as mkf:
    json.dump(temp, mkf, indent="\t")

# #문제별 코드 수가 100 미만인 코드쌍 및 문제 설명 삭제
# removed_num=0
# code_num=0
# with open('nl_pl_p_num.json', 'r') as fff:
#     p_num = json.load(fff)
#     temp={}
#     for i in range(JSON_LEN):
#         new_json=[]
#         print(i)
#         with open(CODE_PAIR_PATH+str(i)+'.json', 'r') as f:
#             json_data = json.load(f)
#             for j in range(len(json_data)):
#                 code_pair=json_data[j]
#                 p_id=code_pair[0]["problem_id"]
#                 if p_num[p_id]>=100:
#                     new_json.append(code_pair)
#                     code_num+=1
#                 else:
#                     if os.path.exists(DESC_PATH+p_id+".html"):
#                         os.remove(DESC_PATH+p_id+".html")
#                     removed_num+=1
#         with open(CODE_PAIR_PATH+str(i)+'.json', 'w',encoding='utf-8') as mkf:
#             json.dump(new_json, mkf, indent="\t")
#     print(code_num,removed_num)
# #630357개의 코드쌍 남김


In [ ]:
# 토큰 길이별 분류
code_num=0
temp={}
for i in range(116):
    new_json=[]
    with open('tmp/'+str(i)+'.json', 'r') as f:
        json_data = json.load(f)
        for j in range(len(json_data)):
            code_pair=json_data[j]
            code_num+=1
            longer=0
            for k in range(2):
                code=code_pair[k]["code_tokens"]
                token=tokenizer.tokenize(code)
                longer=max(len(token),longer)
            try:
                temp[longer]+=1
            except:
                temp[longer]=1
        print(code_num)

with open('tokenlen.json', 'w',encoding='utf-8') as mkf:
    json.dump(temp, mkf, indent="\t")
print("total"+str(code_num))

In [ ]:
#코드쌍이 존재하지 않는 문제 설명 삭제
p_list=os.listdir("p_desc")
total=len(p_list)
with open('nl_pl_p_num.json', 'r') as fff:
    p_num = json.load(fff)
    exists_code=list(p_num.keys())
    print(exists_code)
    remove=0
    for i in p_list:
        if i.split(".")[0] not in exists_code:
            remove+=1
            print(i)
            os.remove("p_desc/"+i)
    print(total,remove,total-remove)

In [ ]:
#코드 쌍의 차이가 큰 코드쌍 제거
p_num=0
for i in range(1):
    new_json=[]
    diff = difflib.Differ()
    with open('tmp/'+str(i)+'.json', 'r') as f:
        json_data = json.load(f)
        diff_value=[]
        for j in range(100):
            code_pair=json_data[j]
            diff_value.append(len(list(diff.compare(code_pair[0]["code_tokens"],code_pair[1]["code_tokens"]))))
            # output = cd.difference(code_pair[0]["code_tokens"],code_pair[1]["code_tokens"],lang = "python")
            # print(output.edit_script())
    print(diff_value)